In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [5]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [6]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [7]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [8]:
##########################################
# --- Agente 1: Agente Traductor --- #
##########################################
def agente_buscador(topico):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction = """
        Eres un asistente de interpretacion medica. Tu tarea es traducir palabras y frases usando la herramienta de busqueda de google (google_search).
        Realizas interpretaciones en Español e Ingles en el contexto medico.
        Utiliza la terminologia medica correcta en ambos idiomas
        Mantén el estilo y tono del texto original en la traducción.
        cunado escriba una palabra me proporcionaras su traduccion al ingles o al español, asegurandote de que la traduccion sea precisa dentro del contexto de la interpretacion medica
        siempre las respuestas seran en el siguiente formato
        Ejemplos de respuesta
        Entrada: Aneurysm
        Salida esperada: Aneurysm / Aneurísma
        ----------------------
        Entrada: Paro cardíaco
        Salida esperada: Paro cardíaco / Cardiac arrest
        ----------------------
        Entrada: Palsy
        Salida esperada: Cerebral Palsy / Parálisis cerebral
        ----------------------
        Entrada: Painkiller/ pain reliever
        Salida esperada: Painkiller/ pain reliever / Analgésico
        """ ,
        description="Agente que busca traducciones de palabras en Google",
        tools=[google_search]
    )

    # These lines were moved inside the function definition
    entrada_do_agente_buscador = f"Tópico: {topico}"
    # Executa o agente
    resposta = call_agent(buscador, entrada_do_agente_buscador)
    return resposta

In [9]:
################################################
# --- Agente 2: Sugiere Vocabulario --- #
################################################
def agente_planejador(topico_2):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Eres un asistente de interpretacion medica. Tu tarea es sugerir vocabulario relacionado con una tematica en especifico.
        El vocabulario sugerido muestra una lista de minimo 10 palabras y hasta 20 palabras traducidas en ingles y espanol que esten directamente relacionas con la tematica solicitada.
        La busqueda se hace con herramienta de busqueda de google (google_search).
        Siempre reponderas en el siguiente formato
        Ejemplo de una lista de vocabulario de una tematica:
        Entrada: Diabetes
        Salida esperada: Hello! We can provide an interpretation for diabetes.
        Here's a list of common diabetes-related terms in English and Spanish:
        Diabetes - Diabetes
        Blood sugar - Azúcar en la sangre / Glucosa en la sangre
        Insulin - Insulina
        Hyperglycemia - Hiperglucemia
        Hypoglycemia - Hipoglucemia
        A1C test - Prueba de A1C / Prueba de hemoglobina glicosilada
        Type 1 diabetes - Diabetes tipo 1
        Type 2 diabetes - Diabetes tipo 2
        Gestational diabetes - Diabetes gestacional
        Glucose meter - Glucómetro
        Insulin injection - Inyección de insulina
        Insulin pump - Bomba de insulina
        Oral medication - Medicamento oral
        Diet - Dieta
        Exercise - Ejercicio
        Neuropathy - Neuropatía
        Retinopathy - Retinopatía
        Nephropathy - Nefropatía
        --------------------------------------------------------
        Entrada: Female Reproductive System
        Salida esperada: Hello! We can provide an interpretation for female reproductive system.
        Here's a list of common female reproductive system-related terms in English and Spanish:
        Female Reproductive system - Sistema reproductivo femenino
        birth canal - canal de parto
        cervix - cuello uterino
        clitoris - clitoris
        eggs - ovulos
        endometrium - endometrio
        fallopian tubes - trompas de falopio
        fundus - fondo del utero
        ovaries - ovarios
        uterine lining - revestimiento uterino
        uterus - utero
        vagina - vagina
        womb - vientre / claustro / matriz
        placenta - placenta

        """,
        description="Agente que busca traducciones de palabras en Google y sugiere vocabulario relacionado",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico_2}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [10]:

print("Welcome to the medical interpretation support system")

# --- Obter o Tópico do Usuário ---
topico = input("❓ What word do you want me to translate?: ")


# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("You forgot to type the topic!")
else:
    print(f"Wonderful! Let's translate then {topico}")
    lancamentos_buscados = agente_buscador(topico)
    print("\n--- 📝 Resultado do Agente 1 (Traductor) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

# Corrected indentation for the while loop and its content
topico = input("❓ What word do you want me to translate?: ")
while topico != "fin":
    print(f"Wonderful! Let's translate then {topico}")
    lancamentos_buscados = agente_buscador(topico)
    print("\n--- 📝 Resultado do Agente 1 (Traductor) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")
    topico = input("❓ What word do you want me to translate?: ") # Added input inside the loop to allow breaking out

Welcome to the medical interpretation support system
❓ What word do you want me to translate?: eye
Wonderful! Let's translate then eye

--- 📝 Resultado do Agente 1 (Traductor) ---



> Eye / Ojo
> 


--------------------------------------------------------------
❓ What word do you want me to translate?: lengua
Wonderful! Let's translate then lengua

--- 📝 Resultado do Agente 1 (Traductor) ---



> Lengua / Tongue
> 


--------------------------------------------------------------
❓ What word do you want me to translate?: gallblader
Wonderful! Let's translate then gallblader

--- 📝 Resultado do Agente 1 (Traductor) ---



> Gallbladder / Vesícula biliar
> 


--------------------------------------------------------------
❓ What word do you want me to translate?: fin


In [12]:
print("Welcome to the medical interpretation support system")

topico_2 = input("❓ What vocabulary do you want me to suggest to you?: ")

# Inserir lógica do sistema de agentes ################################################
if not topico_2:
    print("You forgot to type the topic!")
else:
    print(f"Wonderful! Let's translate and find vocabulary related to {topico_2}")
    plano_de_post = agente_planejador(topico_2)
    print("\n--- 📝 Resultado do Agente 2 (Sugiere Vocabulario) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")
topico_2 = input("❓ What vocabulary do you want me to suggest to you?: ")
while topico_2 != "fin":
    print(f"Wonderful! Let's translate and find vocabulary related to {topico_2}")
    plano_de_post = agente_planejador(topico_2)
    plano_de_post = agente_planejador(topico_2)
    print("\n--- 📝 Resultado do Agente 2 (Sugiere Vocabulario) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")
    topico_2 = input("❓ What vocabulary do you want me to suggest to you?: ")


Welcome to the medical interpretation support system
❓ What vocabulary do you want me to suggest to you?: diabetes
Wonderful! Let's translate and find vocabulary related to diabetes

--- 📝 Resultado do Agente 2 (Sugiere Vocabulario) ---



> Hola! Podemos proporcionar una interpretación para la diabetes.
> Aquí hay una lista de términos comunes relacionados con la diabetes en inglés y español:
> Diabetes - Diabetes
> Blood sugar - Azúcar en la sangre / Glucosa en la sangre
> Insulin - Insulina
> Hyperglycemia - Hiperglucemia
> Hypoglycemia - Hipoglucemia
> A1C test - Prueba de A1C / Prueba de hemoglobina glicosilada
> Type 1 diabetes - Diabetes tipo 1
> Type 2 diabetes - Diabetes tipo 2
> Gestational diabetes - Diabetes gestacional
> Glucose meter - Glucómetro
> Insulin injection - Inyección de insulina
> Insulin pump - Bomba de insulina
> Oral medication - Medicamento oral
> Diet - Dieta
> Exercise - Ejercicio
> Neuropathy - Neuropatía
> Retinopathy - Retinopatía
> Nephropathy - Nefropatía
> 
> Hola! Podemos proporcionar una interpretación para la diabetes.
> Aquí hay una lista de términos comunes relacionados con la diabetes en inglés y español:
> 
> *   Diabetes - Diabetes
> *   Blood sugar - Azúcar en la sangre / Glucosa en la sangre
> *   Insulin - Insulina
> *   Hyperglycemia - Hiperglucemia
> *   Hypoglycemia - Hipoglucemia
> *   A1C test - Prueba de A1C / Prueba de hemoglobina glicosilada
> *   Type 1 diabetes - Diabetes tipo 1
> *   Type 2 diabetes - Diabetes tipo 2
> *   Gestational diabetes - Diabetes gestacional
> *   Glucose meter - Glucómetro
> *   Insulin injection - Inyección de insulina
> *   Insulin pump - Bomba de insulina
> *   Oral medication - Medicamento oral
> *   Diet - Dieta
> *   Exercise - Ejercicio
> *   Neuropathy - Neuropatía
> *   Retinopathy - Retinopatía
> *   Nephropathy - Nefropatía
> *   Beta cells - Las células beta
> *   Autoimmune - Autoinmune


--------------------------------------------------------------
❓ What vocabulary do you want me to suggest to you?: fin
